In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/olist_orders_dataset.csv", parse_dates=["order_purchase_timestamp",
                                                                  "order_approved_at",
                                                                  "order_delivered_carrier_date",
                                                                  "order_delivered_customer_date",
                                                                  "order_estimated_delivery_date"
                                                                 ]).dropna(how="all")
df.info()
df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04


In [3]:
print(df.order_status.unique())

['delivered' 'invoiced' 'shipped' 'processing' 'unavailable' 'canceled'
 'created' 'approved']


In [4]:
df.order_status.value_counts()

order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64

In [5]:
df.order_status.value_counts(normalize=True) * 100

order_status
delivered      97.020344
shipped         1.113223
canceled        0.628513
unavailable     0.612423
invoiced        0.315765
processing      0.302692
created         0.005028
approved        0.002011
Name: proportion, dtype: float64

In [21]:
#Poměr doručených a zrušených objednávek

total_count = len(df)
delivered_count = (df.order_status == "delivered").sum()
canceled_count = (df.order_status == "canceled").sum()

delivered_pct = (delivered_count / total_count) * 100
canceled_pct = (canceled_count / total_count) * 100

print(f"Delivered: {delivered_count} - {delivered_pct:.2f}%")
print(f"Canceled: {canceled_count} - {canceled_pct:.2f}%")

Delivered: 96478 - 97.02%
Canceled: 625 - 0.63%


In [23]:
#Časový rozsah

min_date = df.order_purchase_timestamp.min()
max_date = df.order_purchase_timestamp.max()

print(f"Data range: {min_date.date()} -> {max_date.date()}")

Data range: 2016-09-04 -> 2018-10-17


In [58]:
#Průměrná doba doručení

#pouze doručené objednávky
delivered_df = df[(df.order_status == "delivered")&
                  (df["order_delivered_customer_date"].notna())
].copy() 

#Rozdíl ve dnech
delivered_df["delivery_time_days"] = (delivered_df.order_delivered_customer_date - delivered_df.order_purchase_timestamp).dt.days.astype(int)

avg_days = delivered_df["delivery_time_days"].mean()
median_days = delivered_df["delivery_time_days"].median()

print(f"Average delivery time: {avg_days:.1f} days")
print(f"Median delivery time: {median_days:.1f} days")

Average delivery time: 12.1 days
Median delivery time: 10.0 days
